### GS Sales Activity

In [1]:
import polars as pl
import pandas as pd
import gc
from datetime import datetime, timedelta,date
from dateutil.relativedelta import relativedelta
import numpy as np
import json

In [2]:
# load variables from JSON
with open('vars_wk.json', 'r') as json_file:
    js = json.load(json_file)
bucket = js['bucket']
dflib = f's3://{bucket}/BIT/dataframes/'

In [3]:
# Utility Functions -
def load(df, lib=dflib):
    globals()[df] = pl.read_parquet(f'{lib}{df}.parquet')

---

In [4]:
# pulling in data for bit weekly.
load('gs_calls_p1')

In [27]:
#period_loop_lookup_dict (PN 5 has no data in the feed so skipping)
pld = {1:'1-MONTH',2:'3-MONTHS',3:'6-MONTHS',4:'12-MONTHS',6:'PQTD',7:'YTD'}

# export location - >
OUT = 's3://vortex-staging-a65ced90/BIT/output/GeoSummary/Monthly/'

In [28]:
# Process & Export - 
for k,v in pld.items():
    temp1 = (
        gs_calls_p1
        .with_columns(
            pl.lit('MONTHLY').alias('ReportType'),
            pl.lit(f'{v}').alias('Period')
        )
    )
    temp1.to_pandas().to_csv(f'{OUT}Monthly_GeoSummary_SalesActivity_P{k}_Feed.txt', sep='|', lineterminator='\r\n',index=False)
    print('Exported Feed : ',k)    

Exported Feed :  1
Exported Feed :  2
Exported Feed :  3
Exported Feed :  4
Exported Feed :  6
Exported Feed :  7
